<a href="https://colab.research.google.com/github/baeksumin/k-project/blob/main/data_preprocessing/backtest.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
#talib 설치
!wget -q http://prdownloads.sourceforge.net/ta-lib/ta-lib-0.4.0-src.tar.gz 2>&1 > /dev/null
!tar xvzf ta-lib-0.4.0-src.tar.gz 2>&1 > /dev/null

import os

os.chdir('ta-lib')  

!./configure --prefix=/usr 2>&1 > /dev/null
!make 2>&1 > /dev/null
!make install 2>&1 > /dev/null

os.chdir('../')

!pip install TA-Lib 2>&1 > /dev/null

./configure: line 4354: /usr/bin/file: No such file or directory


In [2]:
import re
import json
import pytz
import requests
import pandas as pd
from io import BytesIO
from datetime import datetime 
from datetime import timedelta
import pickle
from google.colab import drive
# import talib
import numpy as np

drive.mount('/content/drive')
data_path = '/content/drive/MyDrive/[Shared]K-Project/data/'

Mounted at /content/drive


In [3]:
# 저장된 삼성전자(KR7005930003) 데이터 가져오기

with open(data_path + 'initial_data1.json',"r") as json_file:
    json_data = json.load(json_file) 

df = pd.read_json(json_data['KR7005930003'], orient ='index') 
samsung = df.transpose()
samsung

,TRD_DD,ISU_CD,ISU_NM,TDD_CLSPRC,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,MKTCAP,ACC_TRDVOL,EPS,PER,BPS,PBR,DPS,DVD_YLD
0,2021/11/05,005930,삼성전자,"70,200","71,600","71,600","70,200","419,078,735,010,000","12,667,743","3,841",18.28,"39,406",1.78,"2,994",4.26
1,2021/11/04,005930,삼성전자,"70,600","71,200","71,600","70,500","421,466,648,030,000","11,818,987","3,841",18.38,"39,406",1.79,"2,994",4.24
2,2021/11/03,005930,삼성전자,"70,400","71,700","71,700","70,100","420,272,691,520,000","12,770,428","3,841",18.33,"39,406",1.79,"2,994",4.25
3,2021/11/02,005930,삼성전자,"71,500","70,800","72,200","70,700","426,839,452,325,000","16,812,570","3,841",18.61,"39,406",1.81,"2,994",4.19
4,2021/11/01,005930,삼성전자,"69,900","70,200","70,600","69,900","417,287,800,245,000","11,503,729","3,841",18.20,"39,406",1.77,"2,994",4.28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6711,1995/05/08,005930,삼성전자,"121,000","122,000","122,000","120,500","6,578,606,045,000","91,810",-,-,-,-,-,-
6712,1995/05/06,005930,삼성전자,"122,000","122,000","123,500","122,000","6,632,974,690,000","47,440",-,-,-,-,-,-
6713,1995/05/04,005930,삼성전자,"122,500","124,000","124,500","122,000","6,660,159,012,500","175,590",-,-,-,-,-,-
6714,1995/05/03,005930,삼성전자,"123,500","119,500","126,500","119,500","6,714,527,657,500","382,980",-,-,-,-,-,-


In [4]:
#시간순 재정렬
samsung = samsung.sort_values(by=['TRD_DD'])
samsung.reset_index(drop=True,inplace=True)
samsung['TRD_DD']=pd.to_datetime(samsung['TRD_DD']) #datetime변환

In [5]:
# 일년치 데이터만 추출
samsung01 = samsung.copy()
samsung01 = samsung01[(samsung01['TRD_DD']>='2020-10-21') & (samsung01['TRD_DD']<='2021-10-20')]
samsung01 = samsung01.reset_index(drop = True)
samsung01

,TRD_DD,ISU_CD,ISU_NM,TDD_CLSPRC,TDD_OPNPRC,TDD_HGPRC,TDD_LWPRC,MKTCAP,ACC_TRDVOL,EPS,PER,BPS,PBR,DPS,DVD_YLD
0,2020-10-21,005930,삼성전자,"60,900","61,200","61,500","60,600","363,559,757,295,000","15,703,443","3,166",19.24,"37,528",1.62,"1,416",2.33
1,2020-10-22,005930,삼성전자,"60,100","60,300","60,500","59,800","358,783,931,255,000","14,294,095","3,166",18.98,"37,528",1.60,"1,416",2.36
2,2020-10-23,005930,삼성전자,"60,200","60,400","60,400","59,800","359,380,909,510,000","9,322,232","3,166",19.01,"37,528",1.60,"1,416",2.35
3,2020-10-26,005930,삼성전자,"60,400","60,400","61,100","60,100","360,574,866,020,000","13,268,226","3,166",19.08,"37,528",1.61,"1,416",2.34
4,2020-10-27,005930,삼성전자,"59,800","60,300","60,300","59,500","356,992,996,490,000","17,260,416","3,166",18.89,"37,528",1.59,"1,416",2.37
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
242,2021-10-14,005930,삼성전자,"69,400","69,000","69,800","68,800","414,302,908,970,000","19,520,641","3,841",18.07,"39,406",1.76,"2,994",4.31
243,2021-10-15,005930,삼성전자,"70,100","70,200","71,000","70,000","418,481,756,755,000","18,051,612","3,841",18.25,"39,406",1.78,"2,994",4.27
244,2021-10-18,005930,삼성전자,"70,200","70,200","70,300","69,200","419,078,735,010,000","13,486,391","3,841",18.28,"39,406",1.78,"2,994",4.26
245,2021-10-19,005930,삼성전자,"70,600","70,800","71,000","70,400","421,466,648,030,000","9,507,991","3,841",18.38,"39,406",1.79,"2,994",4.24


In [6]:
#인풋 데이터 모양 맞춰주기(backtest에 들어갈 데이터 모양)
samsung_bt = samsung01[['TRD_DD','TDD_OPNPRC','TDD_HGPRC','TDD_LWPRC','TDD_CLSPRC', 'ACC_TRDVOL']]
samsung_bt['TRD_DD'] = pd.to_datetime(samsung_bt['TRD_DD'])
samsung_bt.rename(columns={'TRD_DD':'Date', 'TDD_OPNPRC':'Open', 'TDD_HGPRC':'High','TDD_LWPRC':'Low','TDD_CLSPRC':'Close', 'ACC_TRDVOL':'Volume'}, inplace=True)
samsung_bt.set_index('Date',drop=True,inplace=True)
samsung_bt['OpenInterest'] =  np.zeros(247, dtype=int)
samsung_bt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/pandas/core/frame.py:4308: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: h

,Open,High,Low,Close,Volume,OpenInterest
Date,,,,,,
2020-10-21,"61,200","61,500","60,600","60,900","15,703,443",0
2020-10-22,"60,300","60,500","59,800","60,100","14,294,095",0
2020-10-23,"60,400","60,400","59,800","60,200","9,322,232",0
2020-10-26,"60,400","61,100","60,100","60,400","13,268,226",0
2020-10-27,"60,300","60,300","59,500","59,800","17,260,416",0
...,...,...,...,...,...,...
2021-10-14,"69,000","69,800","68,800","69,400","19,520,641",0
2021-10-15,"70,200","71,000","70,000","70,100","18,051,612",0
2021-10-18,"70,200","70,300","69,200","70,200","13,486,391",0


In [7]:
#데이터프레임 콤마(,) 제거 그리고 타입 소수로 변환
samsung_bt['Open'] = samsung_bt['Open'].str.replace(',','').astype('float')
samsung_bt['High'] = samsung_bt['High'].str.replace(',','').astype('float')
samsung_bt['Low'] = samsung_bt['Low'].str.replace(',','').astype('float')
samsung_bt['Close'] = samsung_bt['Close'].str.replace(',','').astype('float')
samsung_bt['Volume'] = samsung_bt['Volume'].str.replace(',','').astype('float')
samsung_bt

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

,Open,High,Low,Close,Volume,OpenInterest
Date,,,,,,
2020-10-21,61200.0,61500.0,60600.0,60900.0,15703443.0,0
2020-10-22,60300.0,60500.0,59800.0,60100.0,14294095.0,0
2020-10-23,60400.0,60400.0,59800.0,60200.0,9322232.0,0
2020-10-26,60400.0,61100.0,60100.0,60400.0,13268226.0,0
2020-10-27,60300.0,60300.0,59500.0,59800.0,17260416.0,0
...,...,...,...,...,...,...
2021-10-14,69000.0,69800.0,68800.0,69400.0,19520641.0,0
2021-10-15,70200.0,71000.0,70000.0,70100.0,18051612.0,0
2021-10-18,70200.0,70300.0,69200.0,70200.0,13486391.0,0


In [8]:
samsung_bt = samsung_bt.replace(0, -1)
samsung_bt

,Open,High,Low,Close,Volume,OpenInterest
Date,,,,,,
2020-10-21,61200.0,61500.0,60600.0,60900.0,15703443.0,-1
2020-10-22,60300.0,60500.0,59800.0,60100.0,14294095.0,-1
2020-10-23,60400.0,60400.0,59800.0,60200.0,9322232.0,-1
2020-10-26,60400.0,61100.0,60100.0,60400.0,13268226.0,-1
2020-10-27,60300.0,60300.0,59500.0,59800.0,17260416.0,-1
...,...,...,...,...,...,...
2021-10-14,69000.0,69800.0,68800.0,69400.0,19520641.0,-1
2021-10-15,70200.0,71000.0,70000.0,70100.0,18051612.0,-1
2021-10-18,70200.0,70300.0,69200.0,70200.0,13486391.0,-1


In [9]:
!pip install backtrader

     |████████████████████████████████| 410 kB 14.7 MB/s 


In [11]:
import backtrader as bt
from datetime import datetime

# time period 를 5~25로 설정하고 실행

for i in range(5, 26):
    try:
        class firstStrategy(bt.Strategy):
          def __init__(self):
            self.rsi = bt.indicators.RSI_SMA(self.data.close, period = i)

          def next(self):
            if not self.position:
              if self.rsi < 30:
                  self.buy(size=100)
              else:
                if self.rsi > 70:
                  self.sell(size=100)


        #초기 금액 설정
        startcash = 10000

        #Cerebro 인스턴스 생성
        cerebro = bt.Cerebro()

        #Cerebro 에 설정한 전략 추가
        cerebro.addstrategy(firstStrategy)

        #데이터 가져오기
        data = bt.feeds.PandasData(dataname=samsung_bt)
        #data = bt.feeds.YahooFinanceData(dataname='MSFT', fromdate=datetime(2011, 1, 1), todate=datetime(2012, 12, 31))
        #데이터를 Cerebro에 추가
        cerebro.adddata(data)

        #초기금액 세팅
        cerebro.broker.setcash(startcash)

        #Cerebro실행 (전략, 데이터 추가한 후)
        cerebro.run()

        #결과(전략을 통해 얻을수 있는 금액)
        portvalue = cerebro.broker.getvalue()

        #차익계산
        pnl = portvalue - startcash

        # 수익률 계산
        earning = ((portvalue - startcash) / startcash) * 100

        #결과출력
        print('time period: {}'.format(i))
        print('Final Portfolio Value: ${}'.format(portvalue))
        print('P/L: ${}'.format(pnl))
        print('earning rates: {}%'.format(earning))
        print('------------------------------------------------------')

        #차트출력
        # cerebro.plot(style='candlestick')
    except:
        print(i, 'error')
        print('------------------------------------------------------')

5 error
------------------------------------------------------
time period: 6
Final Portfolio Value: $-970000.0
P/L: $-980000.0
earning rates: -9800.0%
------------------------------------------------------
time period: 7
Final Portfolio Value: $-950000.0
P/L: $-960000.0
earning rates: -9600.0%
------------------------------------------------------
time period: 8
Final Portfolio Value: $-920000.0
P/L: $-930000.0
earning rates: -9300.0%
------------------------------------------------------
time period: 9
Final Portfolio Value: $-920000.0
P/L: $-930000.0
earning rates: -9300.0%
------------------------------------------------------
time period: 10
Final Portfolio Value: $-620000.0
P/L: $-630000.0
earning rates: -6300.0%
------------------------------------------------------
time period: 11
Final Portfolio Value: $-620000.0
P/L: $-630000.0
earning rates: -6300.0%
------------------------------------------------------
time period: 12
Final Portfolio Value: $-320000.0
P/L: $-330000.0
earni

In [14]:
!pip install feature_selection_ga

     |████████████████████████████████| 160 kB 15.4 MB/s 


In [15]:
from sklearn.datasets import make_classification
from sklearn import linear_model
from feature_selection_ga import FeatureSelectionGA, FitnessFunction

X, y = make_classification(n_samples=100, n_features=15, n_classes=3,
                           n_informative=4, n_redundant=1, n_repeated=2,
                           random_state=1)

model = linear_model.LogisticRegression(solver='lbfgs', multi_class='auto')
fsga = FeatureSelectionGA(model,X,y, ff_obj = FitnessFunction())
pop = fsga.generate(100)

print(pop)

# 인코딩은 염색체를 1과 0으로 된 문자열로 나타낸다. 가장 대표적인 방식이 이진 문자열이다.

EVOLVING.......
-- GENERATION 1 --
Evaluated 60 individuals
-- GENERATION 2 --
Evaluated 66 individuals
-- GENERATION 3 --
Evaluated 69 individuals
-- GENERATION 4 --
Evaluated 73 individuals
-- GENERATION 5 --
Evaluated 47 individuals
-- Only the fittest survives --
Best individual is [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], (0.68,)
[[1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], [1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0], [1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 1, 0, 1, 0], [1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0], [1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 1, 1, 0], [1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 1, 0], [1, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0], [0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 1, 1, 0], [1, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1], [0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0], [1, 1, 1, 1, 0, 0, 1, 1, 0, 0, 1, 0, 0, 1, 0], [0, 1, 0, 1, 1, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0], [1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 1, 0, 1, 1, 0],